In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [2]:
mx.context.num_gpus()

0

In [3]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/Merged-update_hourly.csv')
df.fillna(0, inplace=True)

In [4]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,2.90,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0


In [5]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'Time'], infer_datetime_format=True)

In [6]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,...,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0,2010-01-01 00:00:00
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,...,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0,2010-01-01 01:00:00
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,...,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0,2010-01-01 02:00:00
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,...,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0,2010-01-01 03:00:00
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,...,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0,2010-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0,2020-12-31 22:00:00


In [7]:
df.columns

Index(['Time', 'WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'TWS_S25B', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'TWS_S26', 'PUMP_S26', 'MEAN_RAIN', 'timestamp'],
      dtype='object')

In [8]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [9]:
train_df.shape

(77145, 21)

In [10]:
test_df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
77145,10/20/18 09:00,1.06,1.01,0.0,0.0,2.41,1.07,0.0,0.0,0.0,...,1.15,0.0,469.19,0.0,2.61,2.35,1.11,0.0,0.0,2018-10-20 09:00:00
77146,10/20/18 10:00,0.68,0.63,0.0,0.0,2.41,0.71,0.0,0.0,0.0,...,0.82,0.0,0.00,0.0,0.00,2.62,0.72,0.0,0.0,2018-10-20 10:00:00
77147,10/20/18 11:00,0.31,0.32,0.0,0.0,2.41,0.33,0.0,0.0,0.0,...,0.42,0.0,0.00,0.0,0.00,2.62,0.37,0.0,0.0,2018-10-20 11:00:00
77148,10/20/18 12:00,0.20,0.23,0.0,0.0,2.40,0.24,0.0,0.0,0.0,...,0.35,0.0,0.00,0.0,0.00,2.67,0.28,0.0,0.0,2018-10-20 12:00:00
77149,10/20/18 13:00,0.41,0.42,0.0,0.0,2.39,0.42,0.0,0.0,0.0,...,0.48,0.0,0.00,0.0,0.00,2.72,0.45,0.0,0.0,2018-10-20 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.29,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.66,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.04,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.18,0.0,0.0,2020-12-31 22:00:00


In [11]:
train_ds = PandasDataset(
    train_df,
    target = 'TWS_S25B',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'WS_S1', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'TWS_S26', 'PUMP_S26', 'MEAN_RAIN',],
    freq = 'H'
)

In [12]:
test_ds = PandasDataset(
    test_df,
    target = 'TWS_S25B',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'WS_S1', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'TWS_S26', 'PUMP_S26', 'MEAN_RAIN',],
    freq = 'H'
)

#### training

In [43]:
K = 24
n_hours = 24*3

In [44]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=150,
    num_batches_per_epoch=50,
    add_default_callbacks=True,
    weight_decay=1e-06)

estimator = DeepAREstimator(
    prediction_length=K,
    freq="H",
    distr_output = NegativeBinomialOutput(),
    use_feat_dynamic_real=True,
    batch_size=32,
    trainer=trainer,
    dropout_rate=0.4
)

model = estimator.train(train_ds)

  2%|█                                                     | 1/50 [00:03<03:04,  3.77s/it, epoch=1/250, avg_epoch_loss=1.43]
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/gluonts/mx/trainer/_base.py:474: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(


UnboundLocalError: local variable 'epoch_loss' referenced before assignment

#### predicted y

In [31]:
# forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=500)
# forecasts = list(forecast_it)
# tss = list(ts_it)
# evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
# agg_metrics, item_metrics = evaluator(tss, forecasts)

In [32]:
quantiles = ['0.025', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.975']
q50 = 5

In [33]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'TWS_S25B',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'WS_S1', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'TWS_S26', 'PUMP_S26', 'MEAN_RAIN',],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].samples
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[2.75       2.52999997 1.50999999 0.94499999 0.63       0.245
 0.60000002 0.96499997 1.58000004 1.80499995 2.28500009 2.6400001
 2.61999989 2.1099999  1.52999997 1.         0.43000001 0.205
 0.26499999 0.68000001 1.19500005 1.77999997 2.08500004 2.36500001
 2.66000009 2.36500001 1.69000006 1.20500004 0.69499999 0.25999999
 0.215      0.63999999 1.10000002 1.60500002 1.96500003 2.19000006
 2.36999989 2.07500005 1.53999996 1.07000005 0.625      0.13
 0.015      0.15000001 0.685      1.14999998 1.73500001 2.19499993]
(48,)
------
1
[2.75       2.52999997 1.50999999 0.94499999 0.63       0.245
 0.60000002 0.96499997 1.58000004 1.80499995 2.28500009 2.6400001
 2.61999989 2.1099999  1.52999997 1.         0.43000001 0.205
 0.26499999 0.68000001 1.19500005 1.77999997 2.08500004 2.36500001
 2.66000009 2.36500001 1.69000006 1.20500004 0.69499999 0.25999999
 0.215      0.63999999 1.10000002 1.60500002 1.96500003 2.19000006
 2.36999989 2.07500005 1.53999996 1.07000005 0.625      0.13
 0.015     

[2.75       2.52999997 1.50999999 0.94499999 0.63       0.245
 0.60000002 0.96499997 1.58000004 1.80499995 2.28500009 2.6400001
 2.61999989 2.1099999  1.52999997 1.         0.43000001 0.205
 0.26499999 0.68000001 1.19500005 1.77999997 2.08500004 2.36500001
 2.66000009 2.36500001 1.69000006 1.20500004 0.69499999 0.25999999
 0.215      0.63999999 1.10000002 1.60500002 1.96500003 2.19000006
 2.36999989 2.07500005 1.53999996 1.07000005 0.625      0.13
 0.015      0.15000001 0.685      1.14999998 1.73500001 2.19499993
 2.2750001  1.45500004 0.87       0.56999999 0.31       0.47
 1.03499997 1.66999996 2.08500004 2.3599999  2.79500008 2.76999998
 2.00500011 1.47500002 0.91000003 0.495      0.1        0.25999999
 0.83999997 1.33500004 1.64499998 2.21000004 2.4749999  2.57500005
 2.41000009 1.75999999 1.245      0.69999999 0.30500001 0.23
 0.55000001 1.14499998 1.61500001 1.97500002 2.23000002 2.4749999
 2.18499994 1.70000005 1.005      0.54500002 0.095      0.045
 0.215      0.82999998 1.32500

[2.75       2.52999997 1.50999999 0.94499999 0.63       0.245
 0.60000002 0.96499997 1.58000004 1.80499995 2.28500009 2.6400001
 2.61999989 2.1099999  1.52999997 1.         0.43000001 0.205
 0.26499999 0.68000001 1.19500005 1.77999997 2.08500004 2.36500001
 2.66000009 2.36500001 1.69000006 1.20500004 0.69499999 0.25999999
 0.215      0.63999999 1.10000002 1.60500002 1.96500003 2.19000006
 2.36999989 2.07500005 1.53999996 1.07000005 0.625      0.13
 0.015      0.15000001 0.685      1.14999998 1.73500001 2.19499993
 2.2750001  1.45500004 0.87       0.56999999 0.31       0.47
 1.03499997 1.66999996 2.08500004 2.3599999  2.79500008 2.76999998
 2.00500011 1.47500002 0.91000003 0.495      0.1        0.25999999
 0.83999997 1.33500004 1.64499998 2.21000004 2.4749999  2.57500005
 2.41000009 1.75999999 1.245      0.69999999 0.30500001 0.23
 0.55000001 1.14499998 1.61500001 1.97500002 2.23000002 2.4749999
 2.18499994 1.70000005 1.005      0.54500002 0.095      0.045
 0.215      0.82999998 1.32500

[2.75       2.52999997 1.50999999 0.94499999 0.63       0.245
 0.60000002 0.96499997 1.58000004 1.80499995 2.28500009 2.6400001
 2.61999989 2.1099999  1.52999997 1.         0.43000001 0.205
 0.26499999 0.68000001 1.19500005 1.77999997 2.08500004 2.36500001
 2.66000009 2.36500001 1.69000006 1.20500004 0.69499999 0.25999999
 0.215      0.63999999 1.10000002 1.60500002 1.96500003 2.19000006
 2.36999989 2.07500005 1.53999996 1.07000005 0.625      0.13
 0.015      0.15000001 0.685      1.14999998 1.73500001 2.19499993
 2.2750001  1.45500004 0.87       0.56999999 0.31       0.47
 1.03499997 1.66999996 2.08500004 2.3599999  2.79500008 2.76999998
 2.00500011 1.47500002 0.91000003 0.495      0.1        0.25999999
 0.83999997 1.33500004 1.64499998 2.21000004 2.4749999  2.57500005
 2.41000009 1.75999999 1.245      0.69999999 0.30500001 0.23
 0.55000001 1.14499998 1.61500001 1.97500002 2.23000002 2.4749999
 2.18499994 1.70000005 1.005      0.54500002 0.095      0.045
 0.215      0.82999998 1.32500

[2.75       2.52999997 1.50999999 0.94499999 0.63       0.245
 0.60000002 0.96499997 1.58000004 1.80499995 2.28500009 2.6400001
 2.61999989 2.1099999  1.52999997 1.         0.43000001 0.205
 0.26499999 0.68000001 1.19500005 1.77999997 2.08500004 2.36500001
 2.66000009 2.36500001 1.69000006 1.20500004 0.69499999 0.25999999
 0.215      0.63999999 1.10000002 1.60500002 1.96500003 2.19000006
 2.36999989 2.07500005 1.53999996 1.07000005 0.625      0.13
 0.015      0.15000001 0.685      1.14999998 1.73500001 2.19499993
 2.2750001  1.45500004 0.87       0.56999999 0.31       0.47
 1.03499997 1.66999996 2.08500004 2.3599999  2.79500008 2.76999998
 2.00500011 1.47500002 0.91000003 0.495      0.1        0.25999999
 0.83999997 1.33500004 1.64499998 2.21000004 2.4749999  2.57500005
 2.41000009 1.75999999 1.245      0.69999999 0.30500001 0.23
 0.55000001 1.14499998 1.61500001 1.97500002 2.23000002 2.4749999
 2.18499994 1.70000005 1.005      0.54500002 0.095      0.045
 0.215      0.82999998 1.32500

...

[2.75       2.52999997 1.50999999 ... 1.62       0.94       0.51499999]
(901776,)
------
18787
[2.75       2.52999997 1.50999999 ... 0.98500001 0.51999998 0.005     ]
(901824,)
------
18788
[2.75       2.52999997 1.50999999 ... 0.57499999 0.015      0.        ]
(901872,)
------
18789
[2.75       2.52999997 1.50999999 ... 0.01       0.         0.125     ]
(901920,)
------
18790
[2.75       2.52999997 1.50999999 ... 0.         0.14       0.69      ]
(901968,)
------
18791
[2.75       2.52999997 1.50999999 ... 0.16       0.63499999 0.91500002]
(902016,)
------
18792
[2.75       2.52999997 1.50999999 ... 0.66000003 0.995      1.28499997]
(902064,)
------
18793
[2.75       2.52999997 1.50999999 ... 0.93000001 1.245      2.20000005]
(902112,)
------
18794
[2.75       2.52999997 1.50999999 ... 1.30999994 1.97000003 2.43499994]
(902160,)
------
18795
[2.75       2.52999997 1.50999999 ... 2.1500001  2.28500009 2.65499997]
(902208,)
------
18796
[2.75       2.52999997 1.50999999 ... 2.28999996 2

[2.75       2.52999997 1.50999999 ... 1.625      1.19000006 0.815     ]
(905952,)
------
18874
[2.75       2.52999997 1.50999999 ... 1.11500001 0.81       0.52499998]
(906000,)
------
18875
[2.75       2.52999997 1.50999999 ... 0.69       0.505      0.185     ]
(906048,)
------
18876
[2.75       2.52999997 1.50999999 ... 0.495      0.2        0.43000001]
(906096,)
------
18877
[2.75       2.52999997 1.50999999 ... 0.215      0.47499999 0.77999997]
(906144,)
------
18878
[2.75       2.52999997 1.50999999 ... 0.38       0.70499998 0.95499998]
(906192,)
------
18879
[2.75       2.52999997 1.50999999 ... 0.88       0.92000002 1.35500002]
(906240,)
------
18880
[2.75       2.52999997 1.50999999 ... 0.91000003 1.22500002 1.72000003]
(906288,)
------
18881
[2.75       2.52999997 1.50999999 ... 1.26499999 1.73500001 1.77999997]
(906336,)
------
18882
[2.75       2.52999997 1.50999999 ... 1.44500005 1.84000003 1.85000002]
(906384,)
------
18883
[2.75       2.52999997 1.50999999 ... 1.98500001 1

[2.75       2.52999997 1.50999999 ... 1.64499998 1.47000003 1.03999996]
(910128,)
------
18961
[2.75       2.52999997 1.50999999 ... 1.5        1.02499998 0.70499998]
(910176,)
------
18962
[2.75       2.52999997 1.50999999 ... 1.005      0.84500003 0.77999997]
(910224,)
------
18963
[2.75       2.52999997 1.50999999 ... 0.70999998 0.72500002 0.75999999]
(910272,)
------
18964
[2.75       2.52999997 1.50999999 ... 0.77999997 0.76999998 1.11000001]
(910320,)
------
18965
[2.75       2.52999997 1.50999999 ... 0.72500002 0.94999999 1.34000003]
(910368,)
------
18966
[2.75       2.52999997 1.50999999 ... 1.01999998 1.33000004 1.52499998]
(910416,)
------
18967
[2.75       2.52999997 1.50999999 ... 1.42999995 1.72000003 1.80999994]
(910464,)
------
18968
[2.75       2.52999997 1.50999999 ... 1.57000005 2.05500007 2.        ]
(910512,)
------
18969
[2.75       2.52999997 1.50999999 ... 2.02999997 2.07500005 2.22000003]
(910560,)
------
18970
[2.75       2.52999997 1.50999999 ... 1.99000001 2

[2.75       2.52999997 1.50999999 ... 1.37       0.85500002 0.495     ]
(914304,)
------
19048
[2.75       2.52999997 1.50999999 ... 1.10000002 0.60500002 0.185     ]
(914352,)
------
19049
[2.75       2.52999997 1.50999999 ... 0.51999998 0.13       0.15000001]
(914400,)
------
19050
[2.75       2.52999997 1.50999999 ... 0.13       0.09       0.30000001]
(914448,)
------
19051
[2.75       2.52999997 1.50999999 ... 0.16       0.36000001 0.52499998]
(914496,)
------
19052
[2.75       2.52999997 1.50999999 ... 0.33500001 0.57999998 1.005     ]
(914544,)
------
19053
[2.75       2.52999997 1.50999999 ... 0.57499999 0.935      1.45500004]
(914592,)
------
19054
[2.75       2.52999997 1.50999999 ... 1.16499996 1.51499999 1.77499998]
(914640,)
------
19055
[2.75       2.52999997 1.50999999 ... 1.51499999 1.77499998 1.88      ]
(914688,)
------
19056
[2.75       2.52999997 1.50999999 ... 1.86000001 2.05500007 1.89499998]
(914736,)
------
19057
[2.75       2.52999997 1.50999999 ... 2.19000006 2

[2.75       2.52999997 1.50999999 ... 1.45000005 1.05499995 0.69      ]
(918480,)
------
19135
[2.75       2.52999997 1.50999999 ... 1.05999994 0.59500003 0.35499999]
(918528,)
------
19136
[2.75       2.52999997 1.50999999 ... 0.57999998 0.31999999 0.28999999]
(918576,)
------
19137
[2.75       2.52999997 1.50999999 ... 0.375      0.33500001 0.55500001]
(918624,)
------
19138
[2.75       2.52999997 1.50999999 ... 0.38       0.65499997 1.01499999]
(918672,)
------
19139
[2.75       2.52999997 1.50999999 ... 0.52499998 1.22000003 1.25      ]
(918720,)
------
19140
[2.75       2.52999997 1.50999999 ... 0.95499998 1.255      1.57000005]
(918768,)
------
19141
[2.75       2.52999997 1.50999999 ... 1.26999998 1.63499999 1.88999999]
(918816,)
------
19142
[2.75       2.52999997 1.50999999 ... 1.76999998 1.82000005 1.74000001]
(918864,)
------
19143
[2.75       2.52999997 1.50999999 ... 1.85000002 1.78499997 1.53499997]
(918912,)
------
19144
[2.75       2.52999997 1.50999999 ... 1.86000001 1

In [34]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(19168, 48)

In [35]:
test_ds_predict_q50_supervised[0:5]

array([[2.75      , 2.52999997, 1.50999999, 0.94499999, 0.63      ,
        0.245     , 0.60000002, 0.96499997, 1.58000004, 1.80499995,
        2.28500009, 2.6400001 , 2.61999989, 2.1099999 , 1.52999997,
        1.        , 0.43000001, 0.205     , 0.26499999, 0.68000001,
        1.19500005, 1.77999997, 2.08500004, 2.36500001, 2.66000009,
        2.36500001, 1.69000006, 1.20500004, 0.69499999, 0.25999999,
        0.215     , 0.63999999, 1.10000002, 1.60500002, 1.96500003,
        2.19000006, 2.36999989, 2.07500005, 1.53999996, 1.07000005,
        0.625     , 0.13      , 0.015     , 0.15000001, 0.685     ,
        1.14999998, 1.73500001, 2.19499993],
       [2.2750001 , 1.45500004, 0.87      , 0.56999999, 0.31      ,
        0.47      , 1.03499997, 1.66999996, 2.08500004, 2.3599999 ,
        2.79500008, 2.76999998, 2.00500011, 1.47500002, 0.91000003,
        0.495     , 0.1       , 0.25999999, 0.83999997, 1.33500004,
        1.64499998, 2.21000004, 2.4749999 , 2.57500005, 2.41000009,
   

#### actual y

In [36]:
from helper import series_to_supervised, stage_series_to_supervised

In [37]:
test_df_supervised = series_to_supervised(test_df[['TWS_S25B']], n_hours, K)

In [38]:
test_df_supervised.iloc[:, -K:].shape

(19168, 48)

In [39]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [40]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+38),var1(t+39),var1(t+40),var1(t+41),var1(t+42),var1(t+43),var1(t+44),var1(t+45),var1(t+46),var1(t+47)
77217,2.61,2.00,1.54,1.13,0.60,0.24,0.53,1.02,1.65,2.16,...,1.71,1.17,0.62,0.04,-0.11,0.11,0.68,1.37,1.95,2.49
77218,2.00,1.54,1.13,0.60,0.24,0.53,1.02,1.65,2.16,2.58,...,1.17,0.62,0.04,-0.11,0.11,0.68,1.37,1.95,2.49,2.85
77219,1.54,1.13,0.60,0.24,0.53,1.02,1.65,2.16,2.58,2.79,...,0.62,0.04,-0.11,0.11,0.68,1.37,1.95,2.49,2.85,2.82
77220,1.13,0.60,0.24,0.53,1.02,1.65,2.16,2.58,2.79,2.70,...,0.04,-0.11,0.11,0.68,1.37,1.95,2.49,2.85,2.82,2.33
77221,0.60,0.24,0.53,1.02,1.65,2.16,2.58,2.79,2.70,2.15,...,-0.11,0.11,0.68,1.37,1.95,2.49,2.85,2.82,2.33,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96380,1.99,2.03,1.75,1.26,0.84,0.25,-0.09,-0.22,0.02,0.55,...,2.42,2.28,1.78,1.26,0.72,0.14,0.09,0.23,0.75,1.31
96381,2.03,1.75,1.26,0.84,0.25,-0.09,-0.22,0.02,0.55,1.17,...,2.28,1.78,1.26,0.72,0.14,0.09,0.23,0.75,1.31,1.68
96382,1.75,1.26,0.84,0.25,-0.09,-0.22,0.02,0.55,1.17,1.65,...,1.78,1.26,0.72,0.14,0.09,0.23,0.75,1.31,1.68,2.04
96383,1.26,0.84,0.25,-0.09,-0.22,0.02,0.55,1.17,1.65,2.05,...,1.26,0.72,0.14,0.09,0.23,0.75,1.31,1.68,2.04,2.21


#### performance on test set

In [42]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 0.198
RMSE = 0.231
